# Scene

As all of us now, in many sitcoms and Seinfeld specifically the show is a series of scene.  
Many times, two separate scenes are completely different (for example George and Jerry are in the street, next scene they sit together in a restaurant).  
Since we except following sentences will help us better predict, we shouldn't consider different scene as following sentences.  
Moreover, we might think that the sentence timing within the scene might help the prediction, for example we remember Kramer break in to the room and the audience laughing.

After watching enough Seinfeld episodes we now that sometimes we have this _[translation music](https://www.youtube.com/watch?v=EP2GMyQdw78)_ it's not always the case, and we did see different timings of this interlude.

In [1]:
from utils import load_corpus, split_train_test
from compare_models import plot_confusion_matrix, compare_models_roc_curve
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
%load_ext autoreload
%autoreload 2

df = load_corpus()

Loading BokehJS ...

Let's start with finding the time between following sentences, we shall remember that new episodes should have time zero

In [2]:
df.loc[:, 'time_from_prev'] = np.array([0] + [df.start[i] - df.end[i - 1]
                                       if df.episode_num[i] == df.episode_num[i - 1] else 0
                                       for i in range(1, len(df.start))])


In [3]:
p = figure(plot_height=300, tools="pan,box_zoom,reset, save")

hist, edges = np.histogram(df.time_from_prev, bins=np.arange(-0.1,50,0.2))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4)

show(p)

What do we see?  
As expected, most of the following sentences occur after < 1 sec.  
But we are interested in the long tail, it's very weird to see distance of > 40 sec, let's look at them

In [4]:
df.iloc[[j for i in df[(df.time_from_prev > 40)].index for j in [i-1, i]]].head(6)

,character,txt,num_words,avg_word_length,start,end,length,is_funny,laugh_time,episode_num,line_num,episode_name,season,total_lines,global_episode_num,time_from_prev
7147,ELAINE,Sauce me.,2.0,4.000000,1253.711,1255.228,1.517,True,1255.3,21,461,The Smelly Car,4,475,15,0.000
7148,ELAINE,Why do we need BO?,5.0,2.800000,1300.174,1302.299,2.125,False,NaN,21,462,The Smelly Car,4,475,15,44.946
11216,GEORGE,"Well, we'll straighten him out.",5.0,5.400000,423.691,426.385,2.694,True,451.5,8,176,The Barber,5,445,24,0.074
11217,GEORGE,What do you do there all day?,7.0,3.285714,470.904,472.437,1.533,False,NaN,8,177,The Barber,5,445,24,44.519
21258,STEIN,I wanna dedicate this song to a very courageo...,11.0,4.000000,1278.528,1283.827,5.299,True,1284.7,19,441,The Jimmy,6,454,44,0.741
21259,KRAMER,"Hey, got the new Penthouse.",5.0,4.600000,1387.970,1390.368,2.398,False,NaN,19,442,The Jimmy,6,454,44,104.143


After watching some episodes that's look all right.  
It's really a new scene, and it there is that match seconds without speaking.  
  
  
We prefer to have false negative than false positive --> we prefer to have two scenes together than to break in a middle of a scene, so we will pick the threshold a bit high, let's say 5 seconds

In [5]:
df.episode_num.sample(1).values[0]

16

In [6]:
# df.loc[:, ['new_scene']] = df['time_from_prev'] > 5
# # Other than that, if it's a new episode we want new scene, used heuristic for that
# df.loc[(df.time_from_prev == 0) & (df.start <= 2), ['new_scene']] = True

df[(df.episode_num == df.episode_num.sample(1).values[0]) & (df.season == df.season.sample(1).values[0])]

,character,txt,num_words,avg_word_length,start,end,length,is_funny,laugh_time,episode_num,line_num,episode_name,season,total_lines,global_episode_num,time_from_prev
3431,GEORGE,So I went to a fashion show.,7.0,3.142857,0.742,2.491,1.749,True,2.7,13,0,The Pick,4,456,8,0.000
3432,GEORGE,"First of all, the whole concept of modeling i...",10.0,5.272727,2.492,5.862,3.370,False,NaN,13,1,The Pick,4,456,8,0.001
3433,GEORGE,to the fashion industry.,4.0,5.250000,5.938,7.232,1.294,False,NaN,13,2,The Pick,4,456,8,0.076
3434,GEORGE,"Because when these women are around, who's lo...",10.0,4.818182,7.306,10.170,2.864,False,NaN,13,3,The Pick,4,456,8,0.074
3435,GEORGE,I didn't notice any of the clothes. We're all...,11.0,4.416667,10.243,13.141,2.898,True,12.0,13,4,The Pick,4,456,8,0.073
3436,GEORGE,"What are we applauding? ""There's great-lookin...",8.0,5.666667,13.212,15.872,2.660,True,16.5,13,5,The Pick,4,456,8,0.071
3437,GEORGE,This is great.,3.0,4.000000,15.948,17.174,1.226,False,NaN,13,6,The Pick,4,456,8,0.076
3438,GEORGE,I'm glad I'm here with all these great-lookin...,9.0,4.600000,17.250,19.978,2.728,False,NaN,13,7,The Pick,4,456,8,0.076
3439,GEORGE,Anybody can design a shirt.,5.0,4.600000,20.053,21.381,1.328,False,NaN,13,8,The Pick,4,456,8,0.075
3440,GEORGE,It takes talent to get all these girls in one...,13.0,3.857143,21.454,25.991,4.537,False,NaN,13,9,The Pick,4,456,8,0.073


Not fantastic, we still see FP